<a href="https://colab.research.google.com/github/ayumu-1212/tabelog_analysis/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import csv
import uuid
from bs4 import BeautifulSoup
from datetime import datetime as dt
from google.colab import drive
drive.mount('/content/drive')

"""
【概要】
店舗ごとの画像フォルダ分け無し
ページ遷移あり
.
└── data
    ├── train_labels.csv
    └── train
        ├── abb65fac-2f52-433e-bcaa-0ef8e36dc0cf.jpg
        ├── c5aa2b2a-6f5c-45c7-bf42-8852e7654b55.jpg
        ├── (…)

【課題】
- scoreがなかった時の処理 -> 前処理で実装、pandas
- dev, testデータの作成 -> splitとかで実装可能
- データ数が多くなった時の処理 -> 実装済み（1000店舗分）
- 画像1枚1枚に対してスコアが現状 -> 将来的に画像3枚に対するスコアを回帰させることを検討
"""

HEADER = ['img_path','score']
MAX_PAGE = 100
MAX_SHOP = 1000
now_time_s = dt.now().strftime("%Y%m%d%H%M%S")
csv_path = "./drive/MyDrive/Colab Notebooks/CATechAccel/data/train_labels_" + now_time_s + ".csv"
new_imgdir_name = 'train_' + now_time_s
os.mkdir("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name)

with open(csv_path, "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(HEADER)
    shop_index = 0
    for n in range(1, MAX_PAGE+1):
        # ショップ数が十分か判定
        if shop_index >= MAX_SHOP:
            break
        print("{}ページ目開始".format(n))
        response = requests.get('https://tabelog.com/tokyo/rstLst/' + str(n) + "/")
        soup = BeautifulSoup(response.text, 'lxml')
        restaurant_boxes = soup.find_all("div", class_="list-rst__body")
        for box in restaurant_boxes:
            # ショップ数が十分か判定
            if shop_index >= MAX_SHOP:
                break

            # レストラン名取得
            name = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("div", class_="list-rst__rst-name-wrap").h3.a.text

            # レビュースコア取得
            try:
                score = box.find("div", class_="list-rst__contents")\
                    .find("div", class_="list-rst__rst-data")\
                    .find("div", class_="list-rst__rate").p.span.string
            except:
                print("スコアなし：{}".format(name))
                continue

            # サムネ画像3枚取得
            try:
                lis = box.find('div', class_='list-rst__rst-photo')\
                    .find('ul', class_="list-rst__thumb-list")\
                    .find_all('li', class_='list-rst__thumb-item')
            except:
                print("画像なし：{}".format(name))
                continue

            # サムネ画像保存 & csvデータ保存
            for li in lis:
                img = li.p.a.img
                r = requests.get(img['data-lazy'])
                file_name = str(uuid.uuid4())+str('.jpg')
                with open("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name + "/" + file_name, 'wb') as file:
                      file.write(r.content)
                writer.writerow([new_imgdir_name + "/" + file_name, score])

            shop_index += 1
    print("スクレイピング終了")

In [31]:
import requests
import os
import csv
import uuid
import re
from bs4 import BeautifulSoup
from datetime import datetime as dt
from google.colab import drive
drive.mount('/content/drive')

"""
【概要】
店舗ごとの画像フォルダ分け無し
ページ遷移あり
スコアを3.0 - 4.0の間に絞る
.
└── data
    ├── train_labels.csv
    └── train
        ├── abb65fac-2f52-433e-bcaa-0ef8e36dc0cf.jpg
        ├── c5aa2b2a-6f5c-45c7-bf42-8852e7654b55.jpg
        ├── (…)

【課題】
- scoreがなかった時の処理 -> 前処理で実装、pandas
- dev, testデータの作成 -> splitとかで実装可能
- データ数が多くなった時の処理 -> 実装済み（1000店舗分）
- 画像1枚1枚に対してスコアが現状 -> 将来的に画像3枚に対するスコアを回帰させることを検討

【取得データ】
- 3枚の画像
- レビュースコア
- レストラン名
- 最寄り駅
- 最寄駅までの距離
- 昼値段帯
- 夜値段帯
- 営業時間帯
"""

HEADER = ['img_path','score', 'name', 'nearest_station', 'distance_to_station', 'lunch_price_range', 'dinner_price_range']
MAX_PAGE = 100
MAX_SHOP = 1000
now_time_s = dt.now().strftime("%Y%m%d%H%M%S")
csv_path = "./drive/MyDrive/Colab Notebooks/CATechAccel/data/train_scores_" + now_time_s + ".csv"
new_imgdir_name = 'train_minmax' + now_time_s
os.mkdir("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name)

with open(csv_path, "w", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(HEADER)
    shop_index = 0
    for n in range(1, MAX_PAGE+1):
        # ショップ数が十分か判定
        if shop_index >= MAX_SHOP:
            break
        print("{}ページ目開始".format(n))
        response = requests.get('https://tabelog.com/tokyo/rstLst/' + str(n) + "/")
        soup = BeautifulSoup(response.text, 'lxml')
        restaurant_boxes = soup.find_all("div", class_="list-rst__body")
        for box in restaurant_boxes:
            # ショップ数が十分か判定
            if shop_index >= MAX_SHOP:
                break

            # レストラン名取得
            name = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("div", class_="list-rst__rst-name-wrap").h3.a.text

            # レビュースコア取得
            try:
                score = box.find("div", class_="list-rst__contents")\
                    .find("div", class_="list-rst__rst-data")\
                    .find("div", class_="list-rst__rate").p.span.string
            except:
                print("スコアなし：{}".format(name))
                continue

            # レビュースコアを3.0 - 4.0に絞る
            if score == None or float(score) < 3.0 or 4.0 < float(score):
                print("スコアが外れ値：{}, score={}".format(name, score))
                continue

            # サムネ画像3枚取得
            try:
                lis = box.find('div', class_='list-rst__rst-photo')\
                    .find('ul', class_="list-rst__thumb-list")\
                    .find_all('li', class_='list-rst__thumb-item')
            except:
                print("画像なし：{}".format(name))
                continue

            # 最寄駅・駅までの距離取得
            about_station = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("div", class_="list-rst__rst-name-wrap")\
                .find("div", class_="list-rst__area-genre").text

            station_pattern = "\S+駅"
            nearest_station = re.search(station_pattern, about_station).group()

            distance_pattern = "[0-9]+.*m"
            distance_to_station = re.search(distance_pattern, about_station).group()

            # 昼・夜の予算
            lunch_price_range = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("ul", class_="list-rst__info")\
                .find_all("li", class_="list-rst__info-item")[0].span.text

            dinner_price_range = box.find("div", class_="list-rst__contents")\
                .find("div", class_="list-rst__rst-data")\
                .find("ul", class_="list-rst__info")\
                .find_all("li", class_="list-rst__info-item")[1].span.text

            # サムネ画像保存 & csvデータ保存
            for li in lis:
                img = li.p.a.img
                r = requests.get(img['data-lazy'])
                file_name = str(uuid.uuid4())+str('.jpg')
                with open("./drive/MyDrive/Colab Notebooks/CATechAccel/data/" + new_imgdir_name + "/" + file_name, 'wb') as file:
                      file.write(r.content)
                writer.writerow([new_imgdir_name + "/" + file_name, score, name, nearest_station, distance_to_station, lunch_price_range, dinner_price_range])

            shop_index += 1
    print("スクレイピング終了")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1ページ目開始
スコアなし：完全個室居酒屋 八重洲 鳥将
2ページ目開始
3ページ目開始
4ページ目開始
スコアが外れ値：和牛と海鮮 個室居酒屋 陸州や 新宿東口店, score=None
5ページ目開始
スコアなし：あじと 赤羽店
6ページ目開始
7ページ目開始
8ページ目開始
スコアが外れ値：和黒屋, score=None
9ページ目開始
10ページ目開始
11ページ目開始
スコアなし：全席扉付き完全個室 鳥之助 新橋SL広場店
12ページ目開始
スコアなし：焼き鳥＆肉寿司食べ放題 個室居酒屋 鳥物語 上野店
13ページ目開始
14ページ目開始
15ページ目開始
16ページ目開始
17ページ目開始
18ページ目開始
19ページ目開始
20ページ目開始
21ページ目開始
22ページ目開始
スコアが外れ値：韓無量 飯田橋東口店, score=None
スコアが外れ値：個室肉居酒屋 町田屋 町田駅前店, score=None
23ページ目開始
スコアが外れ値：いつもの和食や 代々木店, score=None
24ページ目開始
25ページ目開始
26ページ目開始
27ページ目開始
28ページ目開始
29ページ目開始
30ページ目開始
31ページ目開始
32ページ目開始
スコアが外れ値：@ Kitchen NIHONBASHI, score=None
33ページ目開始
スコアが外れ値：完全個室居酒屋 椿 日本橋店, score=None
34ページ目開始
スコアが外れ値：BAR IJ, score=None
35ページ目開始
36ページ目開始
スコアが外れ値：肉バル肉ブーケ＆チーズ食べ飲み放題 個室 Meat StanD 新宿東口店, score=None
37ページ目開始
38ページ目開始
スコアが外れ値：大阪焼肉・ホルモンふたご 恵比寿南店, score=None
39ページ目開始
40ページ目開始
41ページ目開始
42ページ目開始
スコアなし：大衆酒場 わいけい
43ページ目開始
44ページ目開始
45ページ目